code to predict pv generation

In [18]:
import numpy as np
import pandas as pd
from scipy.stats import norm, gamma
import matplotlib.pyplot as plt
import scipy.stats as stats
from matplotlib.ticker import MaxNLocator # To ensure demand axis are integer.


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error


# Third party packages.
import os                       # Execute tasks related to your operating system.

import polars as pl             # Data handling ('Blazingly fast DataFrames') and is a drop-in replacement for pandas.
import pickle                   # Save and load data to and from pickle files.
import time
import json
import os
import joblib

from icecream import ic         # Print variables.


Data Understanding

Load data set. Split with 80% and 20% split

In [19]:
# Load API Key and download directory from config file
CONFIG_FILE = "../config/api-call.json"

def load_config(config_path):
    """Load configuration from a JSON file."""
    with open(config_path, "r") as file:
        return json.load(file)

# Load config values
config = load_config(CONFIG_FILE)

# print(config)
DOWNLOAD_DIR = config["ned"]["ned_download_dir"]
print("DOWNLOAD_DIR :", DOWNLOAD_DIR)
pkl_file_path = f'{DOWNLOAD_DIR}/power-gen-type-0.pkl'

# with open('../data/dc-ned-json-data-1.pkl', 'rb') as pickle_file:
with open(pkl_file_path, 'rb') as pickle_file:
    dc_ned_json_data_1 = pickle.load(pickle_file)

# df_ned_json_data = dc_ned_json_data_1['df_orig']
df_ned_json_data = pd.DataFrame(dc_ned_json_data_1)  # Convert dictionary to DataFrame


# df_ned_json_data = dc_ned_json_data_1['df_orig']
df_ned_json_data = pd.DataFrame(dc_ned_json_data_1)  # Convert dictionary to DataFrame

df_pd_orig = df_ned_json_data
# Show a sample of first 3 rows.
df_pd_orig.head(3)


DOWNLOAD_DIR : ../data/powergen/


,@id,@type,id,point,type,granularity,granularitytimezone,activity,classification,capacity,volume,percentage,emission,emissionfactor,validfrom,validto,lastupdate
0,/v1/utilizations/23791261744,Utilization,23791261744,/v1/points/0,/v1/types/0,/v1/granularities/4,/v1/granularity_time_zones/0,/v1/activities/1,/v1/classifications/2,8134680,2033670,0,0,0,2021-12-31T23:00:00+00:00,2021-12-31T23:15:00+00:00,2024-11-10T19:01:35+00:00
1,/v1/utilizations/23791591475,Utilization,23791591475,/v1/points/0,/v1/types/0,/v1/granularities/4,/v1/granularity_time_zones/0,/v1/activities/1,/v1/classifications/2,8039900,2009975,0,0,0,2021-12-31T23:15:00+00:00,2021-12-31T23:30:00+00:00,2024-11-10T19:01:35+00:00
2,/v1/utilizations/23792244041,Utilization,23792244041,/v1/points/0,/v1/types/0,/v1/granularities/4,/v1/granularity_time_zones/0,/v1/activities/1,/v1/classifications/2,8128668,2032167,0,0,0,2021-12-31T23:30:00+00:00,2021-12-31T23:45:00+00:00,2024-11-10T19:01:35+00:00


Descriptive Statistics

In [20]:
# Using Pandas.
df_pd_num = df_pd_orig.select_dtypes(include='number')

# Column names (Pandas: df_pd_orig_num.columns.tolist()).
l_df_num_names = df_pd_num.columns
print(f"We have {len(l_df_num_names)} numerical variables:\n{l_df_num_names}")

# Convert to datetime format
df_pd_orig_str    = df_pd_orig.select_dtypes(include='object')
print("Number of rows:", df_pd_orig_str.shape[0], "and column names are:", df_pd_orig_str.columns, "and column types are:", df_pd_orig_str.dtypes)

We have 6 numerical variables:
Index(['id', 'capacity', 'volume', 'percentage', 'emission', 'emissionfactor'], dtype='object')
Number of rows: 34944 and column names are: Index(['@id', '@type', 'point', 'type', 'granularity', 'granularitytimezone',
       'activity', 'classification', 'validfrom', 'validto', 'lastupdate'],
      dtype='object') and column types are: @id                    object
@type                  object
point                  object
type                   object
granularity            object
granularitytimezone    object
activity               object
classification         object
validfrom              object
validto                object
lastupdate             object
dtype: object


To check if the output column has unique values

In [21]:
l_col = ['capacity']

for s_col in l_col:
    print(f"{l_col}: {df_pd_orig[l_col]}")
    print(f"{s_col}: {df_pd_orig[s_col].unique()}")

['capacity']:        capacity
0       8134680
1       8039900
2       8128668
3       8090804
4       8101220
...         ...
34939  12094504
34940  12053024
34941  12010416
34942  11933236
34943  11862020

[34944 rows x 1 columns]
capacity: [ 8134680  8039900  8128668 ... 12010416 11933236 11862020]


Contruct a descriptive summary

In [22]:
df_pd_orig.describe()

,id,capacity,volume,percentage,emission,emissionfactor
count,3.494400e+04,3.494400e+04,3.494400e+04,34944.0,34944.0,34944.0
mean,3.763316e+10,9.217292e+06,2.304323e+06,0.0,0.0,0.0
std,7.989161e+09,3.884491e+06,9.711227e+05,0.0,0.0,0.0
min,2.379126e+10,3.050108e+06,7.625270e+05,0.0,0.0,0.0
25%,3.072638e+10,5.972254e+06,1.493064e+06,0.0,0.0,0.0
50%,3.764883e+10,8.605130e+06,2.151282e+06,0.0,0.0,0.0
75%,4.457940e+10,1.195338e+07,2.988344e+06,0.0,0.0,0.0
max,5.152906e+10,2.204448e+07,5.511119e+06,0.0,0.0,0.0


Prepare daya for prediction

In [23]:
# Convert timestamp to datetime format
df_pd_orig["validto"] = pd.to_datetime(df_pd_orig["validto"])

# Extract Features from Timestamp
df_pd_orig["year"] = df_pd_orig["validto"].dt.year
df_pd_orig["month"] = df_pd_orig["validto"].dt.month
df_pd_orig["day"] = df_pd_orig["validto"].dt.day
df_pd_orig["hour"] = df_pd_orig["validto"].dt.hour
df_pd_orig["minute"] = df_pd_orig["validto"].dt.minute

# Select Features (X) and Target Variable (y)
X = df_pd_orig[["year", "month", "day", "hour", "minute"]]  # Features
y = df_pd_orig["capacity"]  # Target (Numerical column to predict)

# Split Data into Training & Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Predict

In [98]:
# Step 4: Train a Model (Linear Regression)


# If you need numeric values (e.g., Unix timestamp or ordinal)
# df_cleaned_datetime_PVW["timestamp"] = df_cleaned_datetime_PVW["datetime_parsed"].astype("int64") // 10**9  # Unix timestamp (seconds since 1970)
# selected_columns_df = df_pd_orig[["validto", "capacity"]]
# 02-02-2025 23:30

# X = selected_columns_df[["validto"]]  # Features
# y = selected_columns_df["capacity"]  # Target variable

'''
model = LinearRegression()
model.fit(X_train, y_train)

#  Step 5: Make Predictions
y_pred = model.predict(X_test)
y_int_pred = np.round(y_pred).astype(int)  # Rounds and converts to int

# Step 6: Evaluate the Model
mae = mean_absolute_error(y_test, y_int_pred)
print(f"Mean Absolute Error: {mae}")

# Step 7: Convert Predictions Back to Polars (Optional)
df_pred = pl.DataFrame({"X Values": X_test.values, "Actual": y_test.values, "Predicted": y_int_pred, "Diff": y_test.values - y_int_pred})
print(df_pred)
'''

'\nmodel = LinearRegression()\nmodel.fit(X_train, y_train)\n\n#  Step 5: Make Predictions\ny_pred = model.predict(X_test)\ny_int_pred = np.round(y_pred).astype(int)  # Rounds and converts to int\n\n# Step 6: Evaluate the Model\nmae = mean_absolute_error(y_test, y_int_pred)\nprint(f"Mean Absolute Error: {mae}")\n\n# Step 7: Convert Predictions Back to Polars (Optional)\ndf_pred = pl.DataFrame({"X Values": X_test.values, "Actual": y_test.values, "Predicted": y_int_pred, "Diff": y_test.values - y_int_pred})\nprint(df_pred)\n'

Predict using LASSO model

In [ ]:
'''
# Train Lasso Regression Model
lasso = Lasso(alpha=0.1)  # Adjust alpha for regularization strength
lasso.fit(X_train, y_train)
# Make Predictions
y_pred = lasso.predict(X_test)
y_int_pred = np.round(y_pred).astype(int)  # Rounds and converts to int

# Evaluate Model
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Step 7: Convert Predictions Back to Polars (Optional)
df_pred = pl.DataFrame({"X Values": X_test.values, "Actual": y_test.values, "Predicted": y_int_pred, "Diff": y_test.values - y_int_pred})
print(df_pred)
'''

Mean Absolute Error: 3196397.141285067
shape: (3_495, 4)
┌──────────────────┬──────────┬───────────┬──────────┐
│ X Values         ┆ Actual   ┆ Predicted ┆ Diff     │
│ ---              ┆ ---      ┆ ---       ┆ ---      │
│ array[i32, 5]    ┆ i64      ┆ i64       ┆ i64      │
╞══════════════════╪══════════╪═══════════╪══════════╡
│ [2022, 5, … 45]  ┆ 8797044  ┆ 9169220   ┆ -372176  │
│ [2022, 2, … 15]  ┆ 13988612 ┆ 9558460   ┆ 4430152  │
│ [2022, 10, … 45] ┆ 8249260  ┆ 8844472   ┆ -595212  │
│ [2022, 11, … 0]  ┆ 5833568  ┆ 8362108   ┆ -2528540 │
│ [2022, 10, … 15] ┆ 10010420 ┆ 8821585   ┆ 1188835  │
│ …                ┆ …        ┆ …         ┆ …        │
│ [2022, 3, … 30]  ┆ 5767792  ┆ 9501241   ┆ -3733449 │
│ [2022, 11, … 45] ┆ 10350588 ┆ 9219634   ┆ 1130954  │
│ [2022, 2, … 30]  ┆ 11813332 ┆ 9718684   ┆ 2094648  │
│ [2022, 10, … 30] ┆ 7231656  ┆ 9029708   ┆ -1798052 │
│ [2022, 7, … 30]  ┆ 15931476 ┆ 8769532   ┆ 7161944  │
└──────────────────┴──────────┴───────────┴──────────┘


Predict using decision tree regression

In [ ]:
'''
# Train Decision Tree Regressor Model
dt_regressor = DecisionTreeRegressor(max_depth=3, random_state=42)
dt_regressor.fit(X_train, y_train)

# Make Predictions
y_pred = dt_regressor.predict(X_test)
y_int_pred = np.round(y_pred).astype(int)  # Rounds and converts to int

# Evaluate Model
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Step 7: Convert Predictions Back to Polars (Optional)
df_pred = pl.DataFrame({"X Values": X_test.values, "Actual": y_test.values, "Predicted": y_int_pred, "Diff": y_test.values - y_int_pred})
print(df_pred)
'''


Mean Absolute Error: 2018878.8761534896
shape: (3_495, 4)
┌──────────────────┬──────────┬───────────┬──────────┐
│ X Values         ┆ Actual   ┆ Predicted ┆ Diff     │
│ ---              ┆ ---      ┆ ---       ┆ ---      │
│ array[i32, 5]    ┆ i64      ┆ i64       ┆ i64      │
╞══════════════════╪══════════╪═══════════╪══════════╡
│ [2022, 5, … 45]  ┆ 8797044  ┆ 5937647   ┆ 2859397  │
│ [2022, 2, … 15]  ┆ 13988612 ┆ 13295208  ┆ 693404   │
│ [2022, 10, … 45] ┆ 8249260  ┆ 6811663   ┆ 1437597  │
│ [2022, 11, … 0]  ┆ 5833568  ┆ 8726592   ┆ -2893024 │
│ [2022, 10, … 15] ┆ 10010420 ┆ 5937647   ┆ 4072773  │
│ …                ┆ …        ┆ …         ┆ …        │
│ [2022, 3, … 30]  ┆ 5767792  ┆ 5937647   ┆ -169855  │
│ [2022, 11, … 45] ┆ 10350588 ┆ 6811663   ┆ 3538925  │
│ [2022, 2, … 30]  ┆ 11813332 ┆ 9895086   ┆ 1918246  │
│ [2022, 10, … 30] ┆ 7231656  ┆ 5937647   ┆ 1294009  │
│ [2022, 7, … 30]  ┆ 15931476 ┆ 13295208  ┆ 2636268  │
└──────────────────┴──────────┴───────────┴──────────┘


Predit using Random Forest regressor

In [26]:
# Train Random Forest Regressor Model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)  # 100 trees in the forest
rf_regressor.fit(X_train, y_train)

# Make Predictions
y_pred = rf_regressor.predict(X_test)
y_int_pred = np.round(y_pred).astype(int)  # Rounds and converts to int

# Evaluate Model
mae = mean_absolute_error(y_test, y_pred)
print(f"Mean Absolute Error: {mae}")

# Print Predictions
df_pred = pl.DataFrame({"X Values": X_test.values, "Actual": y_test.values, "Predicted": y_int_pred, "Diff": y_test.values - y_int_pred})
print(df_pred)

MODEL_DOWNLOAD_DIR = config["ned"]["ned_model_download_dir"]
model_pkl_file_path = f'{MODEL_DOWNLOAD_DIR}power-gen-random-forest-model-0.pkl'
print("model_pkl_file_path :", model_pkl_file_path)

# Save the trained model
joblib.dump(rf_regressor, model_pkl_file_path, compress=3)
print("Model saved successfully!")

Mean Absolute Error: 400517.4624581485
shape: (6_989, 4)
┌──────────────────┬──────────┬───────────┬──────────┐
│ X Values         ┆ Actual   ┆ Predicted ┆ Diff     │
│ ---              ┆ ---      ┆ ---       ┆ ---      │
│ array[i32, 5]    ┆ i64      ┆ i64       ┆ i64      │
╞══════════════════╪══════════╪═══════════╪══════════╡
│ [2022, 5, … 45]  ┆ 8797044  ┆ 8742334   ┆ 54710    │
│ [2022, 2, … 15]  ┆ 13988612 ┆ 13690655  ┆ 297957   │
│ [2022, 10, … 45] ┆ 8249260  ┆ 8440801   ┆ -191541  │
│ [2022, 11, … 0]  ┆ 5833568  ┆ 5954081   ┆ -120513  │
│ [2022, 10, … 15] ┆ 10010420 ┆ 8681210   ┆ 1329210  │
│ …                ┆ …        ┆ …         ┆ …        │
│ [2022, 6, … 30]  ┆ 5938156  ┆ 5813986   ┆ 124170   │
│ [2022, 7, … 0]   ┆ 9519532  ┆ 11431624  ┆ -1912092 │
│ [2022, 6, … 45]  ┆ 6941876  ┆ 7638339   ┆ -696463  │
│ [2022, 6, … 45]  ┆ 8764152  ┆ 10531117  ┆ -1766965 │
│ [2022, 2, … 45]  ┆ 9761800  ┆ 9873844   ┆ -112044  │
└──────────────────┴──────────┴───────────┴──────────┘
model_pk